<a href="https://www.kaggle.com/code/cedricsvrn/h-m-eda-baseline?scriptVersionId=99709696" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

from collections import Counter

transactions = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
sample_submission = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
# /kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv
# /kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv
# /kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv
# /kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv

In [2]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [3]:
sample_submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


# Best sellers recommender

In [4]:
class BestSellersRecsys():
    """FIXME"""
    def __init__(self):
        """FIXME"""
        pass
    
    def fit(self, X):
        """FIXME"""
        self.X_occur = Counter(list(X['article_id']))
        return self
    
    def predict(self, article_id, k=10):
        """FIXME"""
        best_sellers = [article_id for (article_id, occ) in self.X_occur.most_common(k)]
        return best_sellers

In [5]:
bs_recsys = BestSellersRecsys()
bs_recsys.fit(transactions)

In [6]:
bs_recsys.predict([])

[706016001,
 706016002,
 372860001,
 610776002,
 759871002,
 464297007,
 372860002,
 610776001,
 399223001,
 706016003]

# CBF : Cosine Similarity

In [7]:
from math import sqrt

class CosineSimCoOccurenceRecsys():
    """FIXME"""
    def __init__(self):
        """FIXME"""
        pass
    
    def get_l_of_two_from_l(self, l):
        """FIXME"""
        if len(l)>1:
            l_of_two = []
            for i in range(0,len(l)):
                for j in range(i+1,len(l)):
                    if int(l[i])<int(l[j]):
                        l_of_two.append(f"{l[i]}|{l[j]}")
                    else:
                        l_of_two.append(f"{l[j]}|{l[i]}")
            return l_of_two
        return None
    
    def get_frequency(self, article_id, counter_occur_article_id):
        return counter_occur_article_id[int(article_id)]
    
    def get_co_occ_frequency(self, article_id_a, article_id_b, counter_co_occ_article_id):
        if int (article_id_a) < int(article_id_b):
            str_co_occ = article_id_a + '|' + article_id_b
        else:
            str_co_occ = article_id_b + '|' + article_id_a
        return counter_co_occ_article_id[str_co_occ]
    
    def get_cosine_sim(self, freq_a, freq_b, co_occ_freq):
        return (co_occ_freq/sqrt(freq_a*freq_b))
        
    def fit(self, X):
        """FIXME"""
        # Getting frequencies of article_ids
        counter_occur_article_id = Counter(list(X['article_id']))
        
        # Getting baskets
        # FIXME : .head(1000)
        l_baskets = list(X.head(1000).groupby(['t_dat', 'customer_id'])['article_id'].apply(list).reset_index()['article_id'])

        # Getting co-occur frequencies of article_ids
        l_co_occur_article_id = []
        for basket in l_baskets:
            l_of_two = self.get_l_of_two_from_l(basket)
            if l_of_two:
                l_co_occur_article_id += l_of_two

        counter_co_occ_article_id = Counter(l_co_occur_article_id)
        
        # Creating pandas datframe
        # FIXME
        l_article_id_a = [i.split('|', 1)[0] for i in l_co_occur_article_id]
        l_article_id_b = [i.split('|', 1)[1] for i in l_co_occur_article_id]
        df_cosine = pd.DataFrame()
        df_cosine['article_id_a'] = l_article_id_a
        df_cosine['article_id_b'] = l_article_id_b
        df_cosine['freq_a'] = df_cosine['article_id_a'].apply(lambda x : self.get_frequency(x, counter_occur_article_id))
        df_cosine['freq_b'] = df_cosine['article_id_b'].apply(lambda x : self.get_frequency(x, counter_occur_article_id))
        df_cosine['co_occ_freq'] = df_cosine.apply(lambda x: self.get_co_occ_frequency(x.article_id_a, x.article_id_b, counter_co_occ_article_id), axis=1)
        df_cosine['cosine_sim'] = df_cosine.apply(lambda x: self.get_cosine_sim(x.freq_a, x.freq_b, x.co_occ_freq), axis=1)
        df_cosine = df_cosine.sort_values('cosine_sim', ascending=False)
        self.df_cosine = df_cosine
        print(df_cosine.head(20))
        return self
    
    def predict(self, basket, k=10):
        """FIXME"""
        self.df_cosine
        df = self.df_cosine[(self.df_cosine['article_id_a'].isin(basket)) & (~self.df_cosine['article_id_b'].isin(basket))]
        print(df.head())
        print(df[['article_id_b', 'cosine_sim']].groupby(['article_id_b']).mean().head())
        pass

In [8]:
cs_recsys = CosineSimCoOccurenceRecsys().fit(transactions)

     article_id_a article_id_b  freq_a  freq_b  co_occ_freq  cosine_sim
2069    567475004    588155002      43       3            2    0.176090
2070    567475004    588155002      43       3            2    0.176090
2627    566904001    623471001      26       3            1    0.113228
2628    576770001    623471001      27       3            1    0.111111
1244    555470006    583448003      24       6            1    0.083333
1766    372576011    634037002      62      88            6    0.081230
1767    372576011    634037002      62      88            6    0.081230
1772    372576011    634037002      62      88            6    0.081230
1769    372576011    634037002      62      88            6    0.081230
1770    372576011    634037002      62      88            6    0.081230
1771    372576011    634037002      62      88            6    0.081230
2462    615491005    635673002       6      26            1    0.080064
2621    599559001    623471001      54       3            1    0

In [9]:
cs_recsys.predict(['567475004'])

     article_id_a article_id_b  freq_a  freq_b  co_occ_freq  cosine_sim
2069    567475004    588155002      43       3            2    0.176090
2070    567475004    588155002      43       3            2    0.176090
2058    567475004    567594001      43     165            2    0.023744
2059    567475004    567594001      43     165            2    0.023744
2065    567475004    653567001      43     572            2    0.012753
              cosine_sim
article_id_b            
567594001       0.023744
588155002       0.176090
653567001       0.012753


In [10]:
cs_recsys.fit(transactions)

     article_id_a article_id_b  freq_a  freq_b  co_occ_freq  cosine_sim
2069    567475004    588155002      43       3            2    0.176090
2070    567475004    588155002      43       3            2    0.176090
2627    566904001    623471001      26       3            1    0.113228
2628    576770001    623471001      27       3            1    0.111111
1244    555470006    583448003      24       6            1    0.083333
1766    372576011    634037002      62      88            6    0.081230
1767    372576011    634037002      62      88            6    0.081230
1772    372576011    634037002      62      88            6    0.081230
1769    372576011    634037002      62      88            6    0.081230
1770    372576011    634037002      62      88            6    0.081230
1771    372576011    634037002      62      88            6    0.081230
2462    615491005    635673002       6      26            1    0.080064
2621    599559001    623471001      54       3            1    0

In [11]:
transactions

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [12]:
counter_occur_article_id = Counter(list(transactions['article_id']))

In [13]:
counter_occur_article_id['663713001']

0

In [14]:
# counter_occur_article_id

In [15]:
# counter_occur_article_id

In [16]:
# Getting baskets
l_baskets = list(transactions.head(1000).groupby(['t_dat', 'customer_id'])['article_id'].apply(list).reset_index()['article_id'])

# Getting list of two from baskets
def get_l_of_two_from_l(l):
    """FIXME"""
    if len(l)>1:
        l_of_two = []
        for i in range(0,len(l)):
            for j in range(i+1,len(l)):
                if int(l[i])<int(l[j]):
                    l_of_two.append(f"{l[i]}|{l[j]}")
                else:
                    l_of_two.append(f"{l[j]}|{l[i]}")
        return l_of_two
    return None

l_co_occur_article_id = []
for basket in l_baskets:
    l_of_two = get_l_of_two_from_l(basket)
    if l_of_two:
        l_co_occur_article_id += l_of_two

# Getting Counter of co-occur
counter_co_occ_article_id = Counter(l_co_occur_article_id)

In [17]:
l_co_occur_article_id[:10]

['541518023|663713001',
 '505221004|685687003',
 '505221004|685687004',
 '505221004|685687001',
 '505221001|505221004',
 '685687003|685687004',
 '685687001|685687003',
 '505221001|685687003',
 '685687001|685687004',
 '505221001|685687004']

In [18]:
[i.split('|', 1)[0] for i in l_co_occur_article_id][:10]

['541518023',
 '505221004',
 '505221004',
 '505221004',
 '505221001',
 '685687003',
 '685687001',
 '505221001',
 '685687001',
 '505221001']

In [19]:
[i.split('|', 1)[1] for i in l_co_occur_article_id][:10]

['663713001',
 '685687003',
 '685687004',
 '685687001',
 '505221004',
 '685687004',
 '685687003',
 '685687003',
 '685687004',
 '685687004']

In [20]:
# counter_co_occ_article_id

# Evaluating recsys

In [21]:
def is_predicted(customer_id, article_id, model, k):
    predictions = model.predict(article_id, k)
    if article_id in predictions:
        return 1
    else:
        return 0

In [22]:
def history_basket_performance(X, model, k=10):
    """FIXME"""
    X['predicted'] = X.apply(lambda x: is_predicted(x.customer_id, x.article_id, model, k), axis=1)
    is_predicted_count = X[X['predicted']==1].shape[0]
    count = X.shape[0]
    
    return (count, is_predicted_count/count)

In [23]:
history_basket_performance(transactions.head(1000), bs_recsys)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(1000, 0.004)